In [1]:
import nltk
from nltk.corpus import sentence_polarity
import random

In [2]:
sentences = sentence_polarity.sents()
print(len(sentences))
print(type(sentences))
print(sentence_polarity.categories())

for sent in sentences[:5]:
    print(sent)

10662
<class 'nltk.corpus.reader.util.ConcatenatedCorpusView'>
['neg', 'pos']
['simplistic', ',', 'silly', 'and', 'tedious', '.']
["it's", 'so', 'laddish', 'and', 'juvenile', ',', 'only', 'teenage', 'boys', 'could', 'possibly', 'find', 'it', 'funny', '.']
['exploitative', 'and', 'largely', 'devoid', 'of', 'the', 'depth', 'or', 'sophistication', 'that', 'would', 'make', 'watching', 'such', 'a', 'graphic', 'treatment', 'of', 'the', 'crimes', 'bearable', '.']
['[garbus]', 'discards', 'the', 'potential', 'for', 'pathological', 'study', ',', 'exhuming', 'instead', ',', 'the', 'skewed', 'melodrama', 'of', 'the', 'circumstantial', 'situation', '.']
['a', 'visually', 'flashy', 'but', 'narratively', 'opaque', 'and', 'emotionally', 'vapid', 'exercise', 'in', 'style', 'and', 'mystification', '.']


In [3]:
pos_sent = sentence_polarity.sents(categories = "pos")
neg_sent = sentence_polarity.sents(categories = "neg")
print(len(pos_sent))

5331


In [4]:
documents = [(sent, cat) for cat in sentence_polarity.categories()
            for sent in sentence_polarity.sents(categories= cat)]

In [5]:
print(documents[0])
print(documents[-1])
random.shuffle(documents)

(['simplistic', ',', 'silly', 'and', 'tedious', '.'], 'neg')
(['provides', 'a', 'porthole', 'into', 'that', 'noble', ',', 'trembling', 'incoherence', 'that', 'defines', 'us', 'all', '.'], 'pos')


In [6]:
all_words_list = [word for sent, cat in documents for word in sent]
all_words = nltk.FreqDist(all_words_list)
word_items = all_words.most_common(2000)
word_features = [word for word, count in word_items]

In [7]:
word_features[:5]

['.', 'the', ',', 'a', 'and']

In [12]:
def document_features(document, word_features):
    document_words = set(document)
    features = {}
    for word in word_features:
            features["V_{}".format(word)] = (word in document_words)
    return features

In [13]:
feature_sets = [(document_features(doc, word_features), cat) for doc, cat in documents]

In [14]:
feature_sets[0]

({'V_.': True,
  'V_the': False,
  'V_,': True,
  'V_a': True,
  'V_and': False,
  'V_of': False,
  'V_to': False,
  'V_is': False,
  'V_in': False,
  'V_that': False,
  'V_it': False,
  'V_as': False,
  'V_but': False,
  'V_with': False,
  'V_film': False,
  'V_this': False,
  'V_for': False,
  'V_its': False,
  'V_an': False,
  'V_movie': True,
  "V_it's": True,
  'V_be': False,
  'V_on': False,
  'V_you': False,
  'V_not': True,
  'V_by': False,
  'V_about': False,
  'V_more': False,
  'V_one': False,
  'V_like': False,
  'V_has': False,
  'V_are': False,
  'V_at': False,
  'V_from': False,
  'V_than': False,
  'V_"': False,
  'V_all': False,
  'V_--': False,
  'V_his': False,
  'V_have': False,
  'V_so': False,
  'V_if': False,
  'V_or': False,
  'V_story': False,
  'V_i': False,
  'V_too': False,
  'V_just': True,
  'V_who': False,
  'V_into': False,
  'V_what': False,
  'V_most': False,
  'V_out': False,
  'V_no': False,
  'V_much': False,
  'V_even': False,
  'V_good': False,
  

In [19]:
test_set, train_set = feature_sets[:1000], feature_sets[1000:]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [21]:
nltk.classify.accuracy(classifier=classifier,gold= test_set)

0.744

In [22]:
classifier.show_most_informative_features(30)

Most Informative Features
            V_engrossing = True              pos : neg    =     19.8 : 1.0
             V_wonderful = True              pos : neg    =     19.8 : 1.0
                V_boring = True              neg : pos    =     18.5 : 1.0
              V_mediocre = True              neg : pos    =     16.2 : 1.0
                 V_waste = True              neg : pos    =     16.2 : 1.0
            V_unexpected = True              pos : neg    =     15.8 : 1.0
              V_captures = True              pos : neg    =     15.8 : 1.0
               V_routine = True              neg : pos    =     15.6 : 1.0
               V_generic = True              neg : pos    =     15.6 : 1.0
             V_inventive = True              pos : neg    =     13.1 : 1.0
          V_refreshingly = True              pos : neg    =     13.1 : 1.0
            V_refreshing = True              pos : neg    =     13.1 : 1.0
                  V_flat = True              neg : pos    =     12.9 : 1.0

In [26]:
# create your own path to the subjclues file
SLpath = "subjclueslen1-HLTEMNLP05.tff"

# import the Subjectivity program as a module to use the function
import subjectivity
SL = subjectivity.readSubjectivity(SLpath)

In [27]:
len(SL.keys())

6885

In [29]:
print(SL['absolute'])
print(SL['shabby'])
print(SL['dog'])

['strongsubj', 'adj', False, 'neutral']
['strongsubj', 'adj', False, 'negative']


KeyError: 'dog'

In [32]:
strength, pos, isStemmed, polarity = SL["terrible"]
print(polarity)

negative


In [33]:
# define features that include word counts of subjectivity words
# negative feature will have number of weakly negative words +
#    2 * number of strongly negative words
# positive feature has similar definition
#    not counting neutral words
def SL_features(document, word_features, SL):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = (word in document_words)
    # count variables for the 4 classes of subjectivity
    weakPos = 0
    strongPos = 0
    weakNeg = 0
    strongNeg = 0
    for word in document_words:
        if word in SL:
            strength, posTag, isStemmed, polarity = SL[word]
            if strength == 'weaksubj' and polarity == 'positive':
                weakPos += 1
            if strength == 'strongsubj' and polarity == 'positive':
                strongPos += 1
            if strength == 'weaksubj' and polarity == 'negative':
                weakNeg += 1
            if strength == 'strongsubj' and polarity == 'negative':
                strongNeg += 1
            features['positivecount'] = weakPos + (2 * strongPos)
            features['negativecount'] = weakNeg + (2 * strongNeg)      
    return features

In [34]:
SL_featuresets = [(SL_features(d, word_features, SL), c) for (d, c) in documents]

In [35]:
train_set, test_set = SL_featuresets[1000:], SL_featuresets[:1000]

In [36]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.753

In [37]:
###  Negation words
# Negation words "not", "never" and "no"
# Not can appear in contractions of the form "doesn't"
for sent in list(sentences)[:50]:
   for word in sent:
     if (word.endswith("n't")):
       print(sent)

# this list of negation words includes some "approximate negators" like hardly and rarely
negationwords = ['no', 'not', 'never', 'none', 'nowhere', 'nothing', 'noone', 'rather', 'hardly', 'scarcely', 'rarely', 'seldom', 'neither', 'nor']

['there', 'is', 'a', 'difference', 'between', 'movies', 'with', 'the', 'courage', 'to', 'go', 'over', 'the', 'top', 'and', 'movies', 'that', "don't", 'care', 'about', 'being', 'stupid']
['a', 'farce', 'of', 'a', 'parody', 'of', 'a', 'comedy', 'of', 'a', 'premise', ',', 'it', "isn't", 'a', 'comparison', 'to', 'reality', 'so', 'much', 'as', 'it', 'is', 'a', 'commentary', 'about', 'our', 'knowledge', 'of', 'films', '.']
['i', "didn't", 'laugh', '.', 'i', "didn't", 'smile', '.', 'i', 'survived', '.']
['i', "didn't", 'laugh', '.', 'i', "didn't", 'smile', '.', 'i', 'survived', '.']
['most', 'of', 'the', 'problems', 'with', 'the', 'film', "don't", 'derive', 'from', 'the', 'screenplay', ',', 'but', 'rather', 'the', 'mediocre', 'performances', 'by', 'most', 'of', 'the', 'actors', 'involved']
['the', 'lack', 'of', 'naturalness', 'makes', 'everything', 'seem', 'self-consciously', 'poetic', 'and', 'forced', '.', '.', '.', "it's", 'a', 'pity', 'that', "[nelson's]", 'achievement', "doesn't", 'match'

In [38]:
# One strategy with negation words is to negate the word following the negation word
#   other strategies negate all words up to the next punctuation
# Strategy is to go through the document words in order adding the word features,
#   but if the word follows a negation words, change the feature to negated word
# Start the feature set with all 2000 word features and 2000 Not word features set to false
def NOT_features(document, word_features, negationwords):
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = False
        features['V_NOT{}'.format(word)] = False
    # go through document words in order
    for i in range(0, len(document)):
        word = document[i]
        if ((i + 1) < len(document)) and ((word in negationwords) or (word.endswith("n't"))):
            i += 1
            features['V_NOT{}'.format(document[i])] = (document[i] in word_features)
        else:
            features['V_{}'.format(word)] = (word in word_features)
    return features

In [39]:
# define the feature sets
NOT_featuresets = [(NOT_features(d, word_features, negationwords), c) for (d, c) in documents]
# show the values of a couple of example features
print(NOT_featuresets[0][0]['V_NOTcare'])
print(NOT_featuresets[0][0]['V_always'])

False
False


In [40]:
train_set, test_set = NOT_featuresets[1000:], NOT_featuresets[:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.788

In [41]:
classifier.show_most_informative_features(30)

Most Informative Features
            V_engrossing = True              pos : neg    =     19.8 : 1.0
             V_wonderful = True              pos : neg    =     19.8 : 1.0
                V_boring = True              neg : pos    =     18.5 : 1.0
                 V_waste = True              neg : pos    =     16.2 : 1.0
              V_mediocre = True              neg : pos    =     16.2 : 1.0
            V_unexpected = True              pos : neg    =     15.8 : 1.0
              V_captures = True              pos : neg    =     15.8 : 1.0
               V_generic = True              neg : pos    =     15.6 : 1.0
               V_routine = True              neg : pos    =     15.6 : 1.0
          V_refreshingly = True              pos : neg    =     13.1 : 1.0
            V_refreshing = True              pos : neg    =     13.1 : 1.0
             V_inventive = True              pos : neg    =     13.1 : 1.0
                    V_90 = True              neg : pos    =     12.9 : 1.0

In [42]:
stopwords = nltk.corpus.stopwords.words("english")

In [44]:
newstopwords = [word for word in stopwords if word not in negationwords]

In [45]:
new_all_words_list = [word for word in all_words_list if word not in newstopwords]

In [46]:
new_all_words = nltk.FreqDist(new_all_words_list)
new_word_items = new_all_words.most_common(2000)
new_word_features = [word for (word,count) in new_word_items]

In [47]:
# define the feature sets
NOT_featuresets = [(NOT_features(d, new_word_featuresatures, negationwords), c) for (d, c) in documents]

In [48]:
new_featuresets = NOT_featuresets

In [49]:
train_set, test_set = new_featuresets[1000:], new_featuresets[:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.786